In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline 

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar pyspark-shell'


In [ ]:
from yaml import load as yaml_load
import os
import sys
import json
sys.path.append('../')
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType, FloatType, IntegerType, DoubleType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import re
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel, GBTClassifier,OneVsRest, OneVsRestModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
spark = SparkSession.builder.master("local[*]").appName("Data cleaning").getOrCreate()

In [ ]:
spark.sparkContext.addPyFile("/home/ml/Documents/crimes_chigaco_spark/notebooks/sparkxgb.zip")

In [ ]:
from sparkxgb import XGBoostEstimator, XGBoostClassificationModel

In [ ]:
from src.data.make_dataset import LoadDataframe
from src.features.build_features import extract_features_classification

In [ ]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    

config_file = "/home/ml/Documents/crimes_chigaco/config/config.yml"
config = _build_configuration(config_file)

In [ ]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')


In [ ]:
df_temp = obj_df_loaded.df_temperature()
df_sky  = obj_df_loaded.df_sky()
df_socio = obj_df_loaded.df_socio()
df_crimes_socio = obj_df_loaded.df_crime_socio()
obj_extract_features_classification = extract_features_classification(config, df_crimes_socio, df_temp, df_sky)

In [ ]:
from pyspark.sql.functions import isnan, when, count, col, isnull

In [ ]:
df_ml = obj_extract_features_classification.extract_feature()

In [ ]:
df_ml.limit(5).toPandas().T

In [ ]:
df_ml.printSchema()

In [ ]:
#df_ml.select([count(when(isnan(c), c)).alias(c) for c in df_ml.columns]).toPandas()

In [ ]:
categoricalColumns = ['domestic']
numericCols = ['year', 'month', 'day', 'hour','minute', 'latitude',
               'longitude', 'isStreet', 'isAV',
               'isBLVD', 'isRD', 'isPL', 'isBROADWAY',
               'isPKWY', 'duree_day', 'minute','dayofmonth',
               'dayofyear','dayofweek','Temperature',
               'pct_housing_crowded',
               'pct_households_below_poverty',
               'pct_age16_unemployed',
               'pct_age25_no_highschool',
               'pct_not_working_age',
               'per_capita_income',
               'hardship_index',
                'Chicago_broken clouds',
                'Chicago_drizzle',
                 'Chicago_few clouds',
                'Chicago_fog',
                 'Chicago_haze',
                 'Chicago_heavy intensity drizzle',
                 'Chicago_heavy intensity rain',
                 'Chicago_heavy snow',
                 'Chicago_light intensity drizzle',
                 'Chicago_light rain',
                 'Chicago_light rain and snow',
                 'Chicago_light snow',
                 'Chicago_mist',
                 'Chicago_moderate rain',
                 'Chicago_overcast clouds',
                 'Chicago_proximity thunderstorm',
                 'Chicago_scattered clouds',
                 'Chicago_sky is clear',
                 'Chicago_snow',
                 'Chicago_thunderstorm',
                 'Chicago_thunderstorm with heavy rain',
                 'Chicago_thunderstorm with light rain',
                 'Chicago_thunderstorm with rain',
                'Chicago_very heavy rain'
]


In [ ]:
rf = RandomForestClassifier(labelCol='label', featuresCol='features')


In [ ]:
df_train, df_test = df_ml.randomSplit([0.7, 0.3])

In [ ]:
%%time
stages = []
stringIndexer_label = StringIndexer(inputCol = 'primary_type', outputCol = 'label').fit(df_train)

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

In [ ]:
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages.append(stringIndexer)
    stages.append(encoder)
stages.append(stringIndexer_label)
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages.append(assembler)
stages.append(rf)
stages.append(labelConverter)

In [ ]:
pipeline_and_model = Pipeline(stages = stages)

In [ ]:
#paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [100, 110 ,200]).addGrid(rf.maxDepth, [10,12,15]).build()
paramGrid = (ParamGridBuilder().addGrid(rf.numTrees, [ 50 ,60])
             .addGrid(rf.maxDepth, [5,8])
             .build())


In [ ]:
crossval = CrossValidator(estimator=pipeline_and_model,
                          estimatorParamMaps=paramGrid,
                          evaluator= MulticlassClassificationEvaluator(),
                          numFolds=10) 

In [ ]:
cvModel = crossval.fit(df_train)

In [ ]:
bestPipeline = cvModel.bestModel

In [ ]:
%%time
df_test_pred = bestPipeline.transform(df_test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
predictions = df_test_pred.select(col("label"), col("prediction"))

In [ ]:
%%time
predictions_ = predictions.cache()

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(predictions_)

In [ ]:
print(accuracy)

In [ ]:
df_test_pred.limit(3).toPandas().T

In [ ]:
df_test_pred.select('label', 'prediction', 'predictedLabel', 'primary_type').show(5)

In [ ]:
bestRFModel = bestPipeline.stages[4]

In [ ]:
bestParams = bestRFModel.extractParamMap()

In [ ]:
bestParams

In [ ]:
stages

In [ ]:
cvModel.avgMetrics

In [ ]:
cvModel.bestModel.stages